# Detección de tipo de semáforo

In [9]:
# !pip install Pillow
# !pip install rawpy
# !pip install imageio
# !pip install yolov5
# !pip install torch torchvision
# !pip install numpy opencv-python
# !pip install roboflow

In [10]:
import os
import cv2
import numpy as np
import shutil

In [11]:
images_folder = 'datasets/cropped_and_labeled/images'
labels_folder = 'datasets/cropped_and_labeled/labels'
final_output_folder_images = 'datasets/cropped_and_labeled_final/images'
final_output_folder_labels = 'datasets/cropped_and_labeled_final/labels'

In [12]:
if not os.path.exists(final_output_folder_images):
    os.makedirs(final_output_folder_images)

if not os.path.exists(final_output_folder_labels):
    os.makedirs(final_output_folder_labels)

Función que nos va a permitir poder tener los rangos de color al momento de analizar una imagen y saber si se encuentra en el rango de rojo, verde o amarillo.

In [13]:
def get_color_ranges():
    # Rojo puede estar en dos rangos debido a cómo se define en el espacio HSV
    lower_red_1 = np.array([0, 100, 100])
    upper_red_1 = np.array([10, 255, 255])
    lower_red_2 = np.array([160, 100, 100])
    upper_red_2 = np.array([180, 255, 255])

    # Verde
    lower_green = np.array([40, 40, 40])
    upper_green = np.array([90, 255, 255])

    # Amarillo
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])

    return [(lower_red_1, upper_red_1), (lower_red_2, upper_red_2), (lower_green, upper_green), (lower_yellow, upper_yellow)]


Función que permite que podamos utilizar la detección en las imagenes que son de semáforos

In [14]:
def detect_traffic_light_color(image):
    # Convertir la imagen a HSV
    hsv_img = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Obtener los rangos de colores
    color_ranges = get_color_ranges()

    # Crear máscaras para cada color
    red_mask_1 = cv2.inRange(hsv_img, color_ranges[0][0], color_ranges[0][1])
    red_mask_2 = cv2.inRange(hsv_img, color_ranges[1][0], color_ranges[1][1])
    green_mask = cv2.inRange(hsv_img, color_ranges[2][0], color_ranges[2][1])
    yellow_mask = cv2.inRange(hsv_img, color_ranges[3][0], color_ranges[3][1])

    # Combinar las máscaras de rojo
    red_mask = red_mask_1 + red_mask_2

    # Calcular la cantidad de píxeles de cada color
    red_count = cv2.countNonZero(red_mask)
    green_count = cv2.countNonZero(green_mask)
    yellow_count = cv2.countNonZero(yellow_mask)

    # Determinar el color predominante
    if red_count > green_count and red_count > yellow_count:
        return 'red'
    elif green_count > red_count and green_count > yellow_count:
        return 'green'
    elif yellow_count > red_count and yellow_count > green_count:
        return 'yellow'
    else:
        return 'unknown'

Procesamos todas las imagenes que fueron detectadas con objetos por YOLOv5 para poder luego determinar las que son semáforos para determinarles que color tiene actualmente para saber si es rojo, verde, amarillo

In [15]:
def process_images():
    images = os.listdir(images_folder)

    for image_filename in images:
        image_path = os.path.join(images_folder, image_filename)
        label_path = os.path.join(labels_folder, image_filename.replace('.jpg', '.txt'))

        with open(label_path, 'r') as label_file:
            label = label_file.read().strip()

        if label == 'traffic light':
            img = cv2.imread(image_path)
            color = detect_traffic_light_color(img)

            if color != 'unknown':
                new_label = f"traffic_light_{color}"
                # Guardar la imagen y el label actualizado
                output_image_path = os.path.join(final_output_folder_images, image_filename)
                output_label_path = os.path.join(final_output_folder_labels, image_filename.replace('.jpg', '.txt'))

                cv2.imwrite(output_image_path, img)
                with open(output_label_path, 'w') as label_file:
                    label_file.write(new_label)
                # print(f"Imagen {image_filename} guardada con etiqueta: {new_label}")
        else:
            # Copiar las imágenes y etiquetas no relacionadas con "traffic light" al nuevo folder
            shutil.copy(image_path, os.path.join(final_output_folder_images, image_filename))
            shutil.copy(label_path, os.path.join(final_output_folder_labels, image_filename.replace('.jpg', '.txt')))
            # print(f"Imagen {image_filename} no es traffic light. Copiada al nuevo pool.")


In [16]:
process_images()